<a href="https://colab.research.google.com/github/JayVeer18/coursera/blob/master/Cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-cg-toolkit' for glob 'nvidia*'
N

In [10]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2020-08-27 09:58:02--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.16.29
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.16.29|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?hSbk3915dNUBHQWK_GYJZc4XSkFuStOWtktZnjxbbDJYg_JWBkXGF20_vcSa7jwMW2krFeeZuqbIJTXvwkiAFveIEVHVHp4CyLe6lUvY0S5RM7kIzC1J2iDgfyRzvyWNy-4EP0KdRIzZG74ZG9gzDYJNXhynBsn95jS6m9zS9751-A4wT9hU26uz68uMK3tbm_hDUUbrLZbfu61K7nE [following]
--2020-08-27 09:58:02--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?hSbk3915dNUBHQWK_GYJZc4XSkFuStOWtktZnjxbbDJYg_JWBkXGF20_vcSa7jwMW2krFeeZuqbIJTXvwkiAFveIEVHVHp4CyLe6lUvY0S5RM7kIzC1J2iDgfy

In [11]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [12]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-4czfz6a4
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-4czfz6a4
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=d5b9c4d0270e2f2d8dbc0439ec2493ec72e200004934abde08eb096845789d5c
  Stored in directory: /tmp/pip-ephem-wheel-cache-7gl0u3iw/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [13]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [14]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define N 10

__global__ void func(int *a, int *b, int *c,int *d) {
    int tid = blockIdx.x;
    if(tid<N)
      d[tid] = (a[tid] * b[tid])+c[tid];
      //printf("%d %d %d %d\n",a[tid],b[tid],c[tid],d[tid]);
}

int main(void){
    int a[N],b[N],c[N],d[N];
    //double d[N];
    int *dev_a,*dev_b,*dev_c,*dev_d;
   // double *dev_d;

    cudaMalloc((void**)&dev_a,N*sizeof(int));
    cudaMalloc((void**)&dev_b,N*sizeof(int));
    cudaMalloc((void**)&dev_c,N*sizeof(int));
    //cudaMalloc((void**)&dev_d,N*sizeof(double));
    cudaMalloc((void**)&dev_d,N*sizeof(int));
    for(int i=0;i<N;i++){
       a[i] = -i*2;
        b[i] = i+3;
        c[i] = i;
    }

    cudaMemcpy(dev_a,a,N*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b,b,N*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(dev_c,c,N*sizeof(int),cudaMemcpyHostToDevice);
    
    func<<<N,1>>>(dev_a,dev_b,dev_c,dev_d);

    cudaError err = cudaMemcpy(d,dev_d,N*sizeof(int),cudaMemcpyDeviceToHost);
    
    if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
    }
    
    int res=0;
    for(int i=0;i<N;i++){
        res+=d[i];
        //printf("%d %d %d\n",a[i],b[i],c[i]);
    }
    printf("result:%ld",res);

  cudaFree(dev_a);
  cudaFree(dev_b);
  cudaFree(dev_c);
  cudaFree(dev_d);
  return 0;
}


result:4294966501


In [16]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define N 10

__global__ void func(int *a, int *b, int *c,int *d) {
    int tid = threadIdx.x+blockIdx.x*blockDim.x;
    while(tid<N){
      d[tid] = (a[tid] * b[tid])+c[tid];
      //printf("%d %d %d %d\n",a[tid],b[tid],c[tid],d[tid]);
      tid+=blockDim.x*gridDim.x;
    }
}

int main(void){
    int a[N],b[N],c[N],d[N];
    //double d[N];
    int *dev_a,*dev_b,*dev_c,*dev_d;
   // double *dev_d;

    cudaMalloc((void**)&dev_a,N*sizeof(int));
    cudaMalloc((void**)&dev_b,N*sizeof(int));
    cudaMalloc((void**)&dev_c,N*sizeof(int));
    //cudaMalloc((void**)&dev_d,N*sizeof(double));
    cudaMalloc((void**)&dev_d,N*sizeof(int));
    for(int i=0;i<N;i++){
       a[i] = -i*2;
        b[i] = i+3;
        c[i] = i;
    }

    cudaMemcpy(dev_a,a,N*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b,b,N*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(dev_c,c,N*sizeof(int),cudaMemcpyHostToDevice);
    
    func<<<N/2,2>>>(dev_a,dev_b,dev_c,dev_d);

    cudaError err = cudaMemcpy(d,dev_d,N*sizeof(int),cudaMemcpyDeviceToHost);
    
    if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
    }
    
    int res=0;
    for(int i=0;i<N;i++){
        res+=d[i];
        //printf("%d %d %d\n",a[i],b[i],c[i]);
    }
    printf("result:%ld",res);

  cudaFree(dev_a);
  cudaFree(dev_b);
  cudaFree(dev_c);
  cudaFree(dev_d);
  return 0;
}

result:4294966501
